# Задача состояла в генерации идеи для грантов основываясь на грантах, которые выйграли. Было решено использовать предобученную модель от Сбера.

In [ ]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.3 MB/s eta 0:00:00


In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 745, done.
remote: Counting objects: 100% (302/302), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 745 (delta 223), reused 193 (delta 179), pack-reused 443
Receiving objects: 100% (745/745), 428.23 KiB | 14.27 MiB/s, done.
Resolving deltas: 100% (462/462), done.


In [ ]:
!pip install transformers==3.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.0/769.0 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 66.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 97.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.8 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=930ca6e40afc2ccb6a9331ce6939bef34adec7d3252b3d70b31f4f6f4bf65c7f
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13.2


In [ ]:
# !pip install transformers

In [ ]:
import os
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer
import chardet
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import json
from sklearn.model_selection import train_test_split

In [ ]:
def read_json(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        return json.load(f)

In [ ]:
text = read_json('/content/drive/MyDrive/Практика DS/data/active_data_g.json')

In [ ]:
text_task=[]

In [ ]:
text[:1]

[{'title': 'Женский хоккейный клуб',
  'status': 'победитель конкурса',
  'requested_amount': 495153.4,
  'co_financing': 846543.4,
  'amount_of_costs': 472715.56,
  'competition': 'Второй конкурс 2022',
  'grant_direction': 'Охрана здоровья граждан, пропаганда здорового образа жизни',
  'application_rating': 74.0,
  'application_number': '22-2-014460',
  'date_of_submission': '14.03.2022',
  'organization': "ОМСКАЯ РЕГИОНАЛЬНАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ 'АССОЦИАЦИЯ ХОККЕИСТОВ ЛЮБИТЕЛЕЙ'",
  'inn': 5507284623,
  'ogrn': 1215500017347,
  'implementation_period_start': '01.09.2022',
  'implementation_period_end': '30.05.2023',
  'aim': 'Создать условия для организации тренировочного и соревновательного процесса для хоккеисток 16-25 лет г.Омска',
  'tasks': ['Организовать для девушек -хоккеисток старше 16 лет регулярный тренировочный и соревновательный процесс.',
   'Организовать и провести мероприятия по популяризации женского хоккея и хоккея для девочек.'],
  'target_groups': ['Девушки -

In [ ]:
for i in text:
  if i['status'] == 'победитель конкурса':
    text_task.append(i['aim'])

In [ ]:
len(text_task)

1883

In [ ]:
text_task = str(text_task)

In [ ]:
ln = len(text_task)

In [ ]:
train=ln//100*70

In [ ]:
text_task[:5]

"['Соз"

In [ ]:
train_per=text_task[:train]

In [ ]:
test_per = text_task[train:]

In [ ]:
with open("test.txt","w", encoding='utf-8') as f:
  f.write(train_per)

In [ ]:
with open("train.txt","w", encoding='utf-8') as f:
  f.write(test_per)

In [ ]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"

In [ ]:
!rm -rf models

In [ ]:
# создание папки для модели, куда будут сохраняться веса модели
!mkdir models/

# Дообучение модели. Текстовые документы находятся в раннее созданных спарсенных данных в файлах train.txt и test.txt. Здесь можно выбирать название модели, которую нужно дообучить, размер блока в который будет упакован датасет после токенизации, кол-во эпох в обучении. В результате модель дообучится и рассчитает метрику удивления модели – перплексию, где чем значение меньше, тем лучше, здесь оно находилось в диапазоне от 11 до 17. 
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
--output_dir=models/weights \
--model_type=gpt2 \
--model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
--do_train \
--train_data_file=test.txt \
--do_eval \
--eval_data_file=valid.txt \
--per_gpu_train_batch_size 1 \
--gradient_accumulation_steps 1 \
--num_train_epochs 5 \
--block_size 100 \
--overwrite_output_dir

/usr/local/lib/python3.8/dist-packages/transformers/modeling_auto.py:796: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Epoch:   0% 0/5 [00:00<?, ?it/s]
Iteration:   0% 0/555 [00:00<?, ?it/s]
Iteration:   0% 1/555 [00:02<23:13,  2.52s/it]
Iteration:   0% 2/555 [00:02<10:06,  1.10s/it]
Iteration:   1% 4/555 [00:02<04:19,  2.12it/s]
Iteration:   1% 6/555 [00:03<02:41,  3.40it/s]
Iteration:   1% 8/555 [00:03<01:57,  4.64it/s]
Iteration:   2% 10/555 [00:03<01:33,  5.81it/s]
Iteration:   2% 12/555 [00:03<01:19,  6.84it/s]
Iteration:   3% 14/555 [00:03<01:10,  7.70it/s]
Iteration:   3% 16/555 [00:03<01:04,  8.40it/s]
Iteration:   3% 18/555 [00:04<01:00,  8.92it/s]
Iteration:   4% 20/555 [00:04<00:58,  9.20it/s]
Iteration:   4% 22/555 [00:04<00:55,  9.54it

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('/content/models/weights')
model = GPT2LMHeadModel.from_pretrained('/content/models/weights')

In [ ]:
text = "Создать условия для организации тренировочного и соревновательного процесса для хоккеисток" 
inpt = tokenizer.encode(text, return_tensors="pt")

In [ ]:
num_return_sequences = 5

In [ ]:
out = model.generate(inpt,
                                   num_return_sequences= num_return_sequences,
                                   max_length=100, 
                                   repetition_penalty=2.0,
                                   do_sample=True,
                                   top_k=50, top_p=0.95,
                                   temperature=0.7)

In [ ]:
for i in range(num_return_sequences):
        print(tokenizer.decode(out[i]))

Создать условия для организации тренировочного и соревновательного процесса для хоккеисток 14-18 лет города Набережные Челны.', 'Оказание комплексной поддержки гражданам, прибывшим с территорий Донбасса в Костромскую область из зоны боевых действий (ДНР), оказавшимся без крыши над головой на территории г/о город Выкса путем оказания срочной специализированной помощи по средствам мобильной связи или посредством получения адресной гуманитарной помощью от жителей городов Липецка до Дебальцево Донецкой народной республики через создание пунктов временного
Создать условия для организации тренировочного и соревновательного процесса для хоккеисток 16-18 лет г. Бердска, занимающихся любительским спортом на территории Приангарья', 'Обучение родителей детей с РАС (расстройствами аутистического спектра) умению концентрироваться при помощи сенсорной системы в условиях инклюзивного пространства "Интеграл", основанного специально создателями проекта «Школа безопасности». Создание условий по улучшени

In [ ]:
with open("generat_degenerat.txt","w", encoding='utf-8') as f:
  f.write(tokenizer.decode(out[1]))